# Overview
- 後処理の組み合わせ

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path
from glob import glob
import plotly
from plotly import express as px
import simdkalman
import optuna
import pyproj

EXP_NAME = "exp003"
NB_NAME = "nb009"

## utils

In [2]:
def get_groundtruth(path: Path) -> pd.DataFrame:
    output_df = pd.DataFrame()
    
    for path in glob(str(path / 'train/*/*/ground_truth.csv')):
        _df = pd.read_csv(path)
        output_df = pd.concat([output_df, _df])
    output_df = output_df.reset_index(drop=True)
    
    _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
    output_df[['t_'+col for col in _columns]] = output_df[_columns]
    output_df = output_df.drop(columns=_columns, axis=1)
    return output_df


def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    
    return dist


def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    # return output_df

## loading dataset

In [3]:
INPUT_DIR = Path("../input/google-smartphone-decimeter-challenge")

base_train = pd.read_csv(INPUT_DIR / 'baseline_locations_train.csv')
base_test = pd.read_csv(INPUT_DIR / 'baseline_locations_test.csv')
sample_sub = pd.read_csv(INPUT_DIR / 'sample_submission.csv')

ground_truth = get_groundtruth(INPUT_DIR)
ground_truth["phone"] = ground_truth["collectionName"] + "_" + ground_truth["phoneName"]

In [4]:
# base_train_truth = base_train.merge(ground_truth, on=["collectionName", "phoneName", "millisSinceGpsEpoch"], how="left")
# base_train_truth.head()

## collection outliers

In [5]:
def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df


def get_outlier_collection(df: pd.DataFrame):
    df = add_distance_diff(df)
    pro_95 = df["dist_next"].mean() + (df["dist_next"].std() * 3)
    pre_95 = df["dist_prev"].mean() + (df["dist_prev"].std() * 3)
    outlier_idx = df[(df["dist_next"] > pro_95) & (df["dist_prev"] > pre_95)].index

    for i in outlier_idx:
        df.loc[i, "latDeg"] = (df.loc[i-1, "latDeg"] + df.loc[i+1, "latDeg"])/2
        df.loc[i, "lngDeg"] = (df.loc[i-1, "lngDeg"] + df.loc[i+1, "lngDeg"])/2

    return df

In [6]:
base_train = get_outlier_collection(base_train)
base_test = get_outlier_collection(base_test)

## kalman filter

In [7]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)


def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
        
    return df

In [8]:
base_train_kf = apply_kf_smoothing(base_train, kf)
base_test_kf = apply_kf_smoothing(base_test, kf)

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

## mean prediction

In [9]:
def make_lerp_data(df):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    org_columns = df.columns
    
    # Generate a combination of time x collection x phone and combine it with the original data (generate records to be interpolated)
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list =df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')
    
    lerp_df = tmp.merge(df, on=['collectionName', 'millisSinceGpsEpoch', 'phoneName'], how='left')
    lerp_df['phone'] = lerp_df['collectionName'] + '_' + lerp_df['phoneName']
    lerp_df = lerp_df.sort_values(['phone', 'millisSinceGpsEpoch'])
    
    # linear interpolation
    lerp_df['interp_latDeg_prev'] = lerp_df['latDeg'].shift(1)
    lerp_df['interp_latDeg_next'] = lerp_df['latDeg'].shift(-1)
    lerp_df['interp_lngDeg_prev'] = lerp_df['lngDeg'].shift(1)
    lerp_df['interp_lngDeg_next'] = lerp_df['lngDeg'].shift(-1)
    lerp_df['interp_phone_prev'] = lerp_df['phone'].shift(1)
    lerp_df['interp_phone_next'] = lerp_df['phone'].shift(-1)
    lerp_df['interp_time_prev'] = lerp_df['millisSinceGpsEpoch'].shift(1)
    lerp_df['interp_time_next'] = lerp_df['millisSinceGpsEpoch'].shift(-1)
    # Leave only records to be interpolated
    lerp_df = lerp_df[(lerp_df['latDeg'].isnull())&(lerp_df['phone']==lerp_df['interp_phone_prev'])&(lerp_df['phone']==lerp_df['interp_phone_next'])].copy()
    # calc lerp
    lerp_df['latDeg'] = lerp_df['interp_latDeg_prev'] + ((lerp_df['interp_latDeg_next'] - lerp_df['interp_latDeg_prev']) * ((lerp_df['millisSinceGpsEpoch'] - lerp_df['interp_time_prev']) / (lerp_df['interp_time_next'] - lerp_df['interp_time_prev']))) 
    lerp_df['lngDeg'] = lerp_df['interp_lngDeg_prev'] + ((lerp_df['interp_lngDeg_next'] - lerp_df['interp_lngDeg_prev']) * ((lerp_df['millisSinceGpsEpoch'] - lerp_df['interp_time_prev']) / (lerp_df['interp_time_next'] - lerp_df['interp_time_prev']))) 
    
    # Leave only the data that has a complete set of previous and next data.
    lerp_df = lerp_df[~lerp_df['latDeg'].isnull()]
    
    return lerp_df[org_columns]


def calc_mean_pred(df, lerp_df):
    '''
    Make a prediction based on the average of the predictions of phones in the same collection.
    '''
    add_lerp = pd.concat([df, lerp_df])
    mean_pred_result = add_lerp.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean().reset_index()
    mean_pred_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch', "phone"]].copy()
    mean_pred_df = mean_pred_df.merge(
        mean_pred_result[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']],
        on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    
    return mean_pred_df

In [10]:
base_train_lerp = make_lerp_data(base_train_kf)
base_train_mean = calc_mean_pred(base_train_kf, base_train_lerp)

base_test_lerp = make_lerp_data(base_test_kf)
base_test_mean = calc_mean_pred(base_test_kf, base_test_lerp)

## removing device

In [11]:
def get_removedevice(input_df: pd.DataFrame, device: str) -> pd.DataFrame:
    input_df["index"] = input_df.index
    input_df = input_df.sort_values(by="millisSinceGpsEpoch")
    input_df.index = input_df["millisSinceGpsEpoch"].values
    
    output_df = pd.DataFrame()
    for _, subdf in input_df.groupby("collectionName"):
        phones = subdf["phoneName"].unique()
        if (len(phones) == 1) or (not device in phones):
            output_df = pd.concat([output_df, subdf])
            continue

        origin_df = subdf.copy()

        _index = subdf["phoneName"] == device
        subdf.loc[_index, "latDeg"] = np.nan
        subdf.loc[_index, "lngDeg"] = np.nan
        subdf = subdf.interpolate(method="index", limit_area="inside")

        _index = subdf["latDeg"].isnull()
        subdf.loc[_index, "latDeg"] = origin_df.loc[_index, "latDeg"].values
        subdf.loc[_index, "lngDeg"] = origin_df.loc[_index, "lngDeg"].values

        output_df = pd.concat([output_df, subdf])

    output_df.index = output_df["index"].values
    output_df = output_df.sort_index()

    del output_df["index"]

    return output_df

In [12]:
base_train_remove = get_removedevice(base_train_mean, "SamsungS20Ultra")
base_test_remove = get_removedevice(base_test_mean, "SamsungS20Ultra")

In [13]:
# base_train_truth = base_train_remove.merge(ground_truth, on=["collectionName", "phoneName", "millisSinceGpsEpoch"], how="left")
# check_score(base_train_truth)

## position shift

In [14]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z


transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt


def compute_dist(oof, gt):
    # oof = pd.read_csv(fname)
    # gt = pd.read_csv(fname2)
    df = oof.merge(gt, on = ['phone','millisSinceGpsEpoch'])
    dst_oof = calc_haversine(df.latDeg, df.lngDeg, df.t_latDeg, df.t_lngDeg)
    scores = pd.DataFrame({'phone': df.phone,'dst': dst_oof})
    scores_grp = scores.groupby('phone')
    d50 = scores_grp.quantile(.50).reset_index()
    d50.columns = ['phone','q50']
    d95 = scores_grp.quantile(.95).reset_index()
    d95.columns = ['phone','q95']
    return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean())/2, d50.merge(d95)


def position_shift(d, a):
    # d = pd.read_csv(fname)
    d = d.copy()
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone', 'millisSinceGpsEpoch'], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng
    
    d.sort_values(['phone', 'millisSinceGpsEpoch'],inplace = True)
    # ffname = 'shifted_' + fname
    # d[sub_columns].to_csv(ffname, index = False)
    return d


def objective(trial):
    a = trial.suggest_uniform('a', -1, 1)
    score, scores = compute_dist(position_shift(base_train_remove, a), ground_truth)
    return score

In [15]:
study = optuna.create_study()
study.optimize(objective, n_trials=50)

[I 2021-06-19 22:47:04,188] A new study created in memory with name: no-name-9a9e9e33-4498-4ffe-a83c-7ddec56a4549
[I 2021-06-19 22:47:08,914] Trial 0 finished with value: 4.0130137580167 and parameters: {'a': -0.0733721763724029}. Best is trial 0 with value: 4.0130137580167.
[I 2021-06-19 22:47:13,518] Trial 1 finished with value: 3.9536256964269634 and parameters: {'a': 0.6603096303091598}. Best is trial 1 with value: 3.9536256964269634.
[I 2021-06-19 22:47:18,249] Trial 2 finished with value: 4.115938289284963 and parameters: {'a': -0.37102339193991707}. Best is trial 1 with value: 3.9536256964269634.
[I 2021-06-19 22:47:23,197] Trial 3 finished with value: 4.0673762552856045 and parameters: {'a': -0.25021589126164323}. Best is trial 1 with value: 3.9536256964269634.
[I 2021-06-19 22:47:28,104] Trial 4 finished with value: 3.941994345524412 and parameters: {'a': 0.38959996819561815}. Best is trial 4 with value: 3.941994345524412.
[I 2021-06-19 22:47:32,889] Trial 5 finished with valu

In [16]:
# best params: {'a': 0.5027147817506713}
study.best_params

{'a': 0.5011741982769846}

In [17]:
base_test_shift = position_shift(base_test_remove, a=study.best_params["a"])
base_test_shift

,collectionName,phoneName,millisSinceGpsEpoch,phone,latDeg,lngDeg,heightAboveWgs84EllipsoidM,x,y,z,xp,xdiff,yp,ydiff,zp,zdiff,dist,xnew,ynew,znew
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,2020-05-15-US-MTV-1_Pixel4,37.416586,-122.082022,63.5,-2.693951e+06,-4.297520e+06,3.854254e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,2020-05-15-US-MTV-1_Pixel4,37.416595,-122.082039,63.5,-2.693952e+06,-4.297519e+06,3.854255e+06,-2.693951e+06,-1.204108,-4.297520e+06,1.663182,3.854254e+06,1.006061,2.286528,-2.693952e+06,-4.297519e+06,3.854255e+06
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,2020-05-15-US-MTV-1_Pixel4,37.416600,-122.082051,63.5,-2.693953e+06,-4.297518e+06,3.854255e+06,-2.693952e+06,-0.786701,-4.297519e+06,0.893094,3.854255e+06,0.442951,1.269930,-2.693952e+06,-4.297518e+06,3.854255e+06
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,2020-05-15-US-MTV-1_Pixel4,37.416602,-122.082058,63.5,-2.693953e+06,-4.297517e+06,3.854255e+06,-2.693953e+06,-0.561706,-4.297518e+06,0.412971,3.854255e+06,0.067404,0.700430,-2.693953e+06,-4.297518e+06,3.854255e+06
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,2020-05-15-US-MTV-1_Pixel4,37.416602,-122.082062,63.5,-2.693953e+06,-4.297517e+06,3.854256e+06,-2.693953e+06,-0.133474,-4.297517e+06,0.230265,3.854255e+06,0.162361,0.311766,-2.693953e+06,-4.297518e+06,3.854255e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,2021-04-29-US-SJC-3_SamsungS20Ultra,37.334542,-121.899395,63.5,-2.683160e+06,-4.310778e+06,3.847018e+06,-2.683160e+06,0.763334,-4.310778e+06,-0.487079,3.847018e+06,-0.013308,0.905595,-2.683160e+06,-4.310778e+06,3.847018e+06
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,2021-04-29-US-SJC-3_SamsungS20Ultra,37.334542,-121.899386,63.5,-2.683159e+06,-4.310779e+06,3.847018e+06,-2.683160e+06,0.711527,-4.310778e+06,-0.296394,3.847018e+06,0.163041,0.787847,-2.683159e+06,-4.310779e+06,3.847018e+06
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,2021-04-29-US-SJC-3_SamsungS20Ultra,37.334545,-121.899378,63.5,-2.683158e+06,-4.310779e+06,3.847018e+06,-2.683159e+06,0.713269,-4.310779e+06,-0.215200,3.847018e+06,0.254621,0.787335,-2.683159e+06,-4.310779e+06,3.847018e+06
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,2021-04-29-US-SJC-3_SamsungS20Ultra,37.334548,-121.899370,63.5,-2.683157e+06,-4.310779e+06,3.847018e+06,-2.683158e+06,0.702239,-4.310779e+06,-0.177238,3.847018e+06,0.289233,0.779878,-2.683158e+06,-4.310779e+06,3.847018e+06


In [18]:
sample_sub["latDeg"] = base_test_shift["latDeg"]
sample_sub["lngDeg"] = base_test_shift["lngDeg"]

sample_sub.to_csv(f"../output/{EXP_NAME}_from_{NB_NAME}.csv", index=None)